#Assignment 3: Text Classification Using the Stanford SST Sentiment Dataset#

## Stanford Sentiment Treebank - Movie Review Classification Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using keras Tokenizer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 kB 36.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 123.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.

In [ ]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [ ]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train = pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test = pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels= pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

In [ ]:
print("X_train size:", len(X_train))
print("X_test size:", len(X_test))
print("total number of reviews:", len(X_train)+len(X_test))
print("y_train size:", len(y_train))

X_train size: 6920
X_test size: 1821
total number of reviews: 8741
y_train size: 6920


In [ ]:
y_train_labels.value_counts()

Positive    3610
Negative    3310
Name: label, dtype: int64

##2.   Preprocess data using keras tokenizer


In [ ]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

### Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

The Stanford Sentiment Treebank (SST) is a well-known and popular for sentiment analysis. There are 8741 movie reviews in total. After the train-test split, there are 6920 reviews used for training, and 1921 reviews for testing. There are 2 types of labels, where 3610 postive labels and 3310 negative labels. 

Buiding a predictive model using this dataset is really useful. It can benefit a lot companies in marketing, politics, news, online shopping, restaurents, social media etc. to monitor customer feedback, evaluate heir products or services, and support marketing strategis by analyzing customer reviews. It is also beneficial for companies or government to monitor public opinions, and detect sentiment shifts in social media trends. It is also useful in the news industry to automate the sentiment analysis of news articles, providing a quick and efficient way to summarize the general tone of an article. Besides, researchers, especially in social science and language, can benefit from it to understand public options, trends and how people generate sentences, and improve the efficiency of processing text. 


## 3. Run at least three prediction models to try to predict the SST sentiment dataset well.

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, LSTM
from tensorflow.keras.models import Sequential

# Use an Embedding layer and LSTM layers
model1 = Sequential()
model1.add(Embedding(10000, 16, input_length=40))
model1.add(LSTM(32, return_sequences=True, dropout=0.2))
model1.add(LSTM(32, dropout=0.2))
model1.add(Flatten())
model1.add(Dense(64, activation='softmax'))
model1.add(Dense(2, activation='softmax'))
model1.summary()

model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history1 = model1.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 40, 16)            160000    
                                                                 
 lstm_2 (LSTM)               (None, 40, 32)            6272      
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 176,834
Trainable params: 176,834
Non-tr

In [ ]:
# Use an Embedding layer and Conv1d layers
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(Conv1D(32, 3, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(2, activation='softmax'))
model2.summary()

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history2 = model2.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 40, 16)            160000    
                                                                 
 conv1d_2 (Conv1D)           (None, 38, 32)            1568      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 2)                 66        
                                                                 
Total params: 161,634
Trainable params: 161,634
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
173/173 [==============================] - 1s 4ms/step - loss: 0.6664 - acc: 0.6149 - val_loss: 0.8472 

In [ ]:
# Use transfer learning with glove embeddings

# Download Glove embedding matrix weights
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
! unzip glove.6B.zip

--2023-04-16 19:24:38--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-16 19:24:38--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-16 19:24:38--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [ ]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

max_words = 10000

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector


Found 400001 word vectors.


In [ ]:
# Use transfer learning with glove embeddings
model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=40))
model3.add(Flatten())
model3.add(Dense(32, activation='relu'))
model3.add(Dense(2, activation='sigmoid'))
model3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 100)           1000000   
                                                                 
 flatten_2 (Flatten)         (None, 4000)              0         
                                                                 
 dense_7 (Dense)             (None, 32)                128032    
                                                                 
 dense_8 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,128,098
Trainable params: 1,128,098
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = False

model3.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history3 = model3.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model3.save_weights('pre_trained_glove_model.h5')


Epoch 1/10
173/173 [==============================] - 1s 3ms/step - loss: 0.6247 - acc: 0.6510 - val_loss: 0.7363 - val_acc: 0.5636
Epoch 2/10
173/173 [==============================] - 0s 3ms/step - loss: 0.5038 - acc: 0.7509 - val_loss: 0.7477 - val_acc: 0.6113
Epoch 3/10
173/173 [==============================] - 0s 3ms/step - loss: 0.4229 - acc: 0.8022 - val_loss: 0.7001 - val_acc: 0.6727
Epoch 4/10
173/173 [==============================] - 0s 3ms/step - loss: 0.3591 - acc: 0.8466 - val_loss: 0.8144 - val_acc: 0.6228
Epoch 5/10
173/173 [==============================] - 0s 3ms/step - loss: 0.3014 - acc: 0.8786 - val_loss: 0.7424 - val_acc: 0.6908
Epoch 6/10
173/173 [==============================] - 0s 3ms/step - loss: 0.2440 - acc: 0.9090 - val_loss: 0.7545 - val_acc: 0.7045
Epoch 7/10
173/173 [==============================] - 0s 3ms/step - loss: 0.1922 - acc: 0.9352 - val_loss: 0.9524 - val_acc: 0.6546
Epoch 8/10
173/173 [==============================] - 0s 3ms/step - loss: 0.

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model1 = model_to_onnx(model1, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model1.onnx", "wb") as f:
    f.write(onnx_model1.SerializeToString())

onnx_model2 = model_to_onnx(model2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model2.SerializeToString())

onnx_model3 = model_to_onnx(model3, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model3.SerializeToString())


In [ ]:
#Generate predictions from X_test data and submit model to competition
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
#Instantiate Competition
mycompetition= ai.Competition(apiurl)

In [ ]:
#Submit Model 1: 
#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index_1 = model1.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels_1 = [y_train.columns[i] for i in prediction_column_index_1]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model1.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels_1) 

# model version 207

57/57 [==============================] - 1s 8ms/step
Insert search tags to help users find your model (optional): hw3_model1
Provide any useful notes about your model (optional): yh3513 team4

Your model has been submitted as model version 207

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
#Submit Model 2: 
#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index_2 = model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels_2 = [y_train.columns[i] for i in prediction_column_index_2]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels_2) 
# model version 208

57/57 [==============================] - 0s 1ms/step
Insert search tags to help users find your model (optional): hw3 model2
Provide any useful notes about your model (optional): yh3513 team4

Your model has been submitted as model version 208

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
#Submit Model 3: 
#-- Generate predicted y values (Model 3)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index_3 = model3.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels_3 = [y_train.columns[i] for i in prediction_column_index_3]

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels_3) 
# model version 209

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): hw3 model3
Provide any useful notes about your model (optional): yh3513 team4

Your model has been submitted as model version 209

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

## 4. Discuss which models performed better and point out relevant hyper-parameter values for successful models.

I tried three models and model 2 has better performance. The model 2 has an embedding layer with dimenstion 16, which defines the dimension of the dense embedding vectors used to represent each word in the input text. Its Conv1D layer uses a kernel of size 3 to perform convolution on the input sequence and 32 filters to generate 32 feature maps. The following GlobalMaxPooling1D layer performs global max pooling on the feature maps generated by the Conv1D layer. The Dense layer has 2 hidden units. Its optimizer is RMSprop with a default learning rate of 0.001. It's trained on 10 epochs with the batch size of 32.

##5. Discuss with my team, fit and submit more models after learning from my team

I decided to try a bidirectional LSTM.

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional

model4 = Sequential()
model4.add(Embedding(10000, 10, input_length=40))
model4.add(Bidirectional(LSTM(32)))
model4.add(Dense(2, activation='softmax'))

model4.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history4 = model4.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 8s 33ms/step - loss: 0.6593 - acc: 0.6138 - val_loss: 0.8057 - val_acc: 0.1561
Epoch 2/10
173/173 [==============================] - 3s 17ms/step - loss: 0.5502 - acc: 0.7133 - val_loss: 0.6185 - val_acc: 0.7262
Epoch 3/10
173/173 [==============================] - 3s 18ms/step - loss: 0.4036 - acc: 0.8248 - val_loss: 0.6303 - val_acc: 0.6553
Epoch 4/10
173/173 [==============================] - 3s 17ms/step - loss: 0.3135 - acc: 0.8689 - val_loss: 0.6527 - val_acc: 0.7341
Epoch 5/10
173/173 [==============================] - 6s 35ms/step - loss: 0.2592 - acc: 0.8974 - val_loss: 1.2220 - val_acc: 0.5542
Epoch 6/10
173/173 [==============================] - 7s 38ms/step - loss: 0.2197 - acc: 0.9126 - val_loss: 0.5112 - val_acc: 0.8006
Epoch 7/10
173/173 [==============================] - 3s 17ms/step - loss: 0.1892 - acc: 0.9267 - val_loss: 0.5753 - val_acc: 0.7673
Epoch 8/10
173/173 [==============================] - 3s 15ms/step - 

In [ ]:
# Save keras model to local ONNX file
onnx_model4 = model_to_onnx(model4, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model4.onnx", "wb") as f:
    f.write(onnx_model4.SerializeToString())

In [ ]:
#Submit Model 4: 

#-- Generate predicted y values (Model 4)
prediction_column_index4 = model4.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels_4 = [y_train.columns[i] for i in prediction_column_index4]

# Submit Model 4 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model4.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels_4)

#model version 215

57/57 [==============================] - 1s 13ms/step
Insert search tags to help users find your model (optional): model 4
Provide any useful notes about your model (optional): 1jiahe team4

Your model has been submitted as model version 215

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

After discussion and learning from my team, I tried bidirectional LSTM model, which preformes better. In the embedding layer, the max words is set to 10000, features to 10, and max length to 40. A bidirectional LSTM layer is followed with 32 hidden nodes. 10 epochs and 32 batch sizes are the same to previous models.

I tried 4 models in total. Model1 uses an Embedding layer and LSTM layers. Model2 uses an Embedding layer and Conv1d layers. Model3 uses transfer learning with glove embeddings. Model4 is a bidirectional LSTM model after discussing and learning from my team. Model 4 has the best performance, with hyperparameters:


*   An embedding layers with max words of 10000, dimension of 10 and input sequence length of 40
*   A bidirectional LSTM layer with 32 hidden 
*   A dense layer outputs a 2-dimensional vector with softmax activation function 
*   The model is trained for 10 epochs with a batch size of 32.






